Reduce the sample size to test the preprocess file.

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
df = pd.read_csv("reddit_hyperlinks.csv")

In [12]:
df = df[:10000]

In [13]:
df.shape

(10000, 93)

In [14]:
df_200 = df[:200]
df_200.to_csv("reddit_hyperlinks_200.csv", index=False)

In [15]:
df.head()

,Unnamed: 0,i,u,ts,f1,f2,f3,f4,f5,f6,...,f80,f81,f82,f83,f84,f85,f86,f87,f88,label
0,0,55,0,1388507998,1.0,0.069470,0.007186,0.006333,0.754070,0.020411,...,0.110405,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0
1,1,5599,1,1388513917,0.0,0.000000,0.001262,0.001246,0.739638,0.023245,...,0.213450,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0
2,2,4502,2,1388588075,1.0,0.000000,0.000874,0.000916,0.750365,0.027626,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0
3,3,44,3,1388511475,1.0,0.023766,0.026099,0.022889,0.770335,0.002060,...,0.066703,0.0,0.0,0.0,0.0,0.0,0.0,0.203125,0.017857,0
4,4,3835,4,1388544673,1.0,0.020110,0.016169,0.014573,0.775902,0.008189,...,0.080044,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.028571,0


In [8]:
newFile = df.to_csv("reddit_hyperlinks_10k.csv", index=False)

In [1]:
from platform import node
from matplotlib import use
import numpy as np
import pandas as pd
import argparse
import sys
import os
from pathlib import Path

def remove_first_two_directories(path):
    path_parts = path.split(os.sep)[3:]  # Split the path and discard the first 3 parts
    new_path = os.sep.join(path_parts)  # Rejoin the remaining parts
    return new_path

# Get the current working directory
full_dir = os.getcwd()
modified_dir = remove_first_two_directories(full_dir)
pruned_dir=Path(modified_dir).parents[4]
p=str("/home/"+str(pruned_dir))
sys.path.append(p)

In [ ]:
def simulate_dataset_train_flag(df):
    labels = df['label'].to_numpy()
    mask = (labels == 1) | (labels == 0)
    return mask

def rename_columns_wiki_reddit(file):
    
    df = pd.read_csv(file, skiprows=1, header=None)
    feat_nums = df.shape[1] - 4
    new_columns = ['u', 'i', 'ts', 'label']
    
    for i in range(feat_nums):
        new_columns.append( f'f{i}' )
    
    rename_dict = {i: new_columns[i] for i in range(len(new_columns))}
    df.rename(columns=rename_dict, inplace=True)
    df.to_csv(file, index=False)
    print(f'rename the columns of {file}.')

def reindex(df):
    df['i'] += df['u'].max() + 1
    df['u'] += 1
    df['i'] += 1
    df['e_idx'] = df.index.values + 1
    df['idx'] = df.e_idx
    return df

In [ ]:
#functions imported from the original code
#Source: from tgnnexplainer.xgraph.dataset.tg_dataset import verify_dataframe_unify, check_wiki_reddit_dataformat

def verify_dataframe_unify(df):
    for col in ['u', 'i', 'ts', 'label', 'e_idx', 'idx']:
        assert col in df.columns.to_list()
    
#    assert df.iloc[:, 0].min() == 1
#    assert df.iloc[:, 0].max() == df.iloc[:, 0].nunique()
#    assert df.iloc[:, 1].min() == df.iloc[:, 0].max() + 1
#    assert df.iloc[:, 1].max() == df.iloc[:, 0].max() + df.iloc[:, 1].nunique()
    assert df['e_idx'].min() == 1
    assert df['e_idx'].max() == len(df)
    assert df['idx'].min() == 1
    assert df['idx'].max() == len(df)
    
def check_wiki_reddit_dataformat(df):
    assert df.iloc[:, 0].min() == 0
#    assert df.iloc[:, 0].max() + 1 == df.iloc[:, 0].nunique() # 0, 1, 2, ...
    assert df.iloc[:, 1].min() == 0
 #   assert df.iloc[:, 1].max() + 1 == df.iloc[:, 1].nunique() # 0, 1, 2, ...
    
    for col in ['u', 'i', 'ts', 'label']:
        assert col in df.columns.to_list()

In [7]:
def run(data_name, out_dir=None):
    # from tgnnexplainer.__init__ import ROOT_DIR
    # from tgnnexplainer.xgraph.dataset.tg_dataset import verify_dataframe_unify, check_wiki_reddit_dataformat

    data_path = '/Users/hetavpatel/Desktop/Data Science/Grad DS Work/DSCI 789 Explainable AI/XAI-Cybersec/XAI-CyberSec/tgnnexplainer/reddit_hyperlinks_10k.csv'

    # PATH = './processed/{}.csv'.format(data_name)
    # if out_dir is None:
    #     out_dir = Path('./processed/')
    # else:
    #     out_dir = Path(out_dir)

    OUT_DF = 'ml_{}.csv'.format(data_name)
    OUT_EDGE_FEAT = 'ml_{}.npy'.format(data_name)
    OUT_NODE_FEAT = 'ml_{}_node.npy'.format(data_name)
    
    df = pd.read_csv(data_path)

    # the very raw format
    if 'comma_separated_list_of_features' in df.columns.tolist():
        rename_columns_wiki_reddit(data_path)
        df = pd.read_csv(data_path)

    check_wiki_reddit_dataformat(df)

    # import ipdb; ipdb.set_trace()
    df = reindex(df)
    verify_dataframe_unify(df)
    
    new_df = df

    # set edge feature and node feature
    if data_name == 'simulate_v2':
        raise NotImplementedError
    elif data_name == 'simulate_v1':
        raise NotImplementedError

    elif data_name == 'wikipedia' or data_name == 'reddit' or data_name == 'mooc' or data_name == 'reddit_v1' or data_name == 'reddit_hyperlinks': # added mooc dataset
        select_columns = [c for c in new_df.columns if 'f' in c] # features
        edge_feat = np.zeros((len(df) + 1, len(select_columns))) # 0-th pad with 0
        edge_feat[1:, :] = new_df[select_columns].to_numpy()

        edge_feat_dim = edge_feat.shape[1]
        num_nodes = new_df.i.max()
        node_feat = np.zeros((num_nodes + 1, edge_feat_dim))

    else: 
        raise NotImplementedError

    assert len(node_feat) == new_df.i.max() + 1
    assert len(edge_feat) == len(new_df) + 1

    print('dataset: ', data_name)
    print('edge feature shape: ', edge_feat.shape)
    print('node feature shape: ', node_feat.shape)
    new_df[['u', 'i', 'ts', 'label', 'idx', 'e_idx']].to_csv(OUT_DF, index=False)
    np.save(OUT_EDGE_FEAT, edge_feat) # edge feature matrix
    np.save(OUT_NODE_FEAT, node_feat) # node feature matrix
    print(f'{OUT_DF} saved')
    print(f'{OUT_EDGE_FEAT} saved')
    print(f'{OUT_NODE_FEAT} saved')


def process_garden_5():
    from tgnnexplainer import ROOT_DIR
    data_dir = ROOT_DIR/'xgraph'/'dataset'/'data'
    data_path = data_dir/'garden_5.csv'
    df = pd.read_csv(data_path)
    if 'label' not in df.columns.to_list():
        df['label'] = np.ones((len(df),))
        df.to_csv(data_path, index=False)


In [8]:
if __name__ == '__main__':
    # parser = argparse.ArgumentParser()
    # parser.add_argument('-d', '--data', type=str, default='simulate')
    # parser.add_argument('-rename_w_r', action='store_true', help='rename columns of wikipedia and reddit')
    # args = parser.parse_args()
    # dataset = args.data

    run("reddit")

dataset:  reddit
edge feature shape:  (10001, 88)
node feature shape:  (23622, 88)
ml_reddit.csv saved
ml_reddit.npy saved
ml_reddit_node.npy saved
